In [ ]:
video_dir = "C:/Users/araya/Desktop/keypoints/video_extract"
actions = []
actions = os.listdir(video_dir)
actions

In [ ]:
import os
import cv2
import numpy as np
import vidaug.augmentors as va

# Define the paths to the input and output video directories
input_dir = 'C:/Users/araya/Desktop/keypoints/video_extract'
output_dir = 'C:/Users/araya/Desktop/augments'

# Loop over each video file in the input directory
for file in actions:
    video_path = os.path.join(input_dir, file)
    
    # Load the video file
    cap = cv2.VideoCapture(video_path)

    # Define the output video writer
    # Meta.
    fps = int(cap.get(cv2.CAP_PROP_FPS))
    width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
    height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
    frame_size = (width, height)
    fourcc = cv2.VideoWriter_fourcc(*'mp4v')
    out_path = os.path.join(output_dir,file)
    # Video writer.
    out = cv2.VideoWriter(out_path, fourcc, fps, frame_size)

    # collect all frames of the video
    frames = []
    while(cap.isOpened()):
        ret, frame = cap.read()
        if ret == True:
            frames.append(frame)
        else:
            break
    cap.release()

    # Apply the video augmentation pipeline to each frame of the video
    sometimes = lambda aug: va.Sometimes(0.5, aug) # Used to apply augmentor with 50% probability
    seq = va.Sequential([sometimes(va.InvertColor()),sometimes(va.Salt()),sometimes(va.Pepper())])
    #augment the frames
    frames=np.array(frames, dtype=np.float64)
    video_aug = seq(frames)
    
    # output the video
    for frame in video_aug:
        out.write(frame)
    out.release()

In [ ]:
len(frames)

In [1]:
import cv2
import numpy as np
from imgaug import augmenters as iaa

# Function to read a video and augment its frames
def augment_video(input_video_path, output_video_path):
    # Open the input video
    cap = cv2.VideoCapture(input_video_path)
    width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
    height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
    fps = int(cap.get(cv2.CAP_PROP_FPS))
    
    # Define the codec and create a VideoWriter object to save the video
    fourcc = cv2.VideoWriter_fourcc(*'XVID')
    out = cv2.VideoWriter(output_video_path, fourcc, fps, (width, height))
    
    # Define augmentation sequence using imgaug
    seq = iaa.Sequential([
        iaa.Fliplr(0.5),  # Horizontally flip 50% of the frames
        iaa.Affine(rotate=(-10, 10)),  # Random rotation between -10 and 10 degrees
        iaa.AddToBrightness((-30, 30)),  # Random brightness change
        iaa.AddToHueAndSaturation((-20, 20)),  # Random hue and saturation change
        iaa.GaussianBlur(sigma=(0.0, 1.0))  # Apply Gaussian blur with random sigma
    ])

    while cap.isOpened():
        ret, frame = cap.read()
        if not ret:
            break
        
        # Convert frame from BGR (OpenCV format) to RGB (imgaug format)
        frame_rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
        
        # Augment the frame
        frame_aug = seq(image=frame_rgb)
        
        # Convert the augmented frame back to BGR for OpenCV
        frame_bgr = cv2.cvtColor(frame_aug, cv2.COLOR_RGB2BGR)
        
        # Write the augmented frame to the output video
        out.write(frame_bgr)
    
    # Release the video capture and writer objects
    cap.release()
    out.release()

# Usage Example
input_video = 'C:/Users/araya/Desktop/keypoints/video_extract/กฎกระทรวง.mp4'   # Path to the input video file
output_video = "C:/Users/araya/Desktop/augments" # Path to save the augmented video
augment_video(input_video, output_video)
